In [2]:
import pandas as pd

# Combine base_md and patients into a single DataFrame
base_md = {
    "Patient_001": {"OD": [-4, -5, -6, -7, -8, -9],
                    "OS": [-3, -4, -5, -6, -7, -8]},
    "Patient_002": {"OD": [-15.51, -15.10, -16.01, -16.55, -19.97, -21.00],
                    "OS": [-13.70, -13.80, -15.01, -16.50, -16.97, -20.00]},
    "Patient_003": {"OD": [-10, -11, -12, -13, -14, -15],
                    "OS": [-9, -10, -11, -12, -13, -14]},
    "Patient_004": {"OD": [-6, -6.5, -7, -8, -9, -9.5],
                    "OS": [-5, -5.5, -6, -7, -8, -8.5]},
    "Patient_005": {"OD": [-12, -13, -14, -15, -16, -17],
                    "OS": [-11, -12, -13, -14, -15, -16]},
}

patients = {
    "Patient_001": {"Sex": "Male",   "Age": 65},
    "Patient_002": {"Sex": "Female", "Age": 72},
    "Patient_003": {"Sex": "Male",   "Age": 58},
    "Patient_004": {"Sex": "Female", "Age": 67},
    "Patient_005": {"Sex": "Male",   "Age": 75},
}

# Flatten the base_md dictionary
flattened_data = []
for patient, eyes in base_md.items():
    for i, (od, os) in enumerate(zip(eyes["OD"], eyes["OS"])):
        flattened_data.append({
            "Patient": patient,
            "Timepoint": i + 1,
            "OD": od,
            "OS": os
        })

# Create a DataFrame from the flattened data
df_base_md = pd.DataFrame(flattened_data)

# Add patient metadata
df_patients = pd.DataFrame.from_dict(patients, orient="index").reset_index()
df_patients.rename(columns={"index": "Patient"}, inplace=True)

# Merge the two DataFrames
df = pd.merge(df_base_md, df_patients, on="Patient")

df

,Patient,Timepoint,OD,OS,Sex,Age
0,Patient_001,1,-4.00,-3.00,Male,65
1,Patient_001,2,-5.00,-4.00,Male,65
2,Patient_001,3,-6.00,-5.00,Male,65
3,Patient_001,4,-7.00,-6.00,Male,65
4,Patient_001,5,-8.00,-7.00,Male,65
5,Patient_001,6,-9.00,-8.00,Male,65
6,Patient_002,1,-15.51,-13.70,Female,72
7,Patient_002,2,-15.10,-13.80,Female,72
8,Patient_002,3,-16.01,-15.01,Female,72
9,Patient_002,4,-16.55,-16.50,Female,72


In [4]:
df.to_csv("/home/fmedeiros/Downloads/glaucoma_interface/data/fake_patients.csv", index=False)

In [6]:
print(df.iloc[0])

Patient      Patient_001
Timepoint              1
OD                  -4.0
OS                  -3.0
Sex                 Male
Age                   65
Name: 0, dtype: object
